In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "name": "TerraAI.ipynb",
      "provenance": [],
      "collapsed_sections": [],
      "authorship_tag": "ABX9TyP5ci1I3OCd3S96aclue0iX",
      "include_colab_link": true
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python"
    }
  },
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "view-in-github",
        "colab_type": "text"
      },
      "source": [
        "<a href=\"https://colab.research.google.com/github/bl146u/stuff/blob/master/TerraAI.ipynb\" target=\"_parent\"><img src=\"https://colab.research.google.com/assets/colab-badge.svg\" alt=\"Open In Colab\"/></a>"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "cellView": "form",
        "id": "PVeN7DN2vERj"
      },
      "source": [
        "#@title Запуск TerraAI\n",
        "\n",
        "import os\n",
        "import shutil\n",
        "import requests\n",
        "\n",
        "from git import Repo\n",
        "from enum import Enum\n",
        "from time import sleep\n",
        "from typing import Tuple\n",
        "from pathlib import Path\n",
        "from google.colab import drive, output\n",
        "from subprocess import Popen, PIPE, STDOUT\n",
        "from ipywidgets import widget, Text, Password, Button, VBox, HBox\n",
        "\n",
        "\n",
        "TERRA_REPOSITORY = \"https://github.com/aiuniver/terra_gui.git\"\n",
        "TERRA_SERVER_API = \"http://%sterra.neural-university.ru/api/v1\"\n",
        "TERRA_DIRECTORY = \"terra\"\n",
        "\n",
        "\n",
        "class WebServerException(Exception):\n",
        "    pass\n",
        "\n",
        "\n",
        "class State(str, Enum):\n",
        "    idle = \"idle\"\n",
        "    launch = \"launch\"\n",
        "\n",
        "\n",
        "def show_exception(message):\n",
        "    print(f\"\\033[1;31m{str(message)}\\033[0m\")\n",
        "\n",
        "\n",
        "def handler_change_field(field):\n",
        "    ws.gui.launch_button.disabled = not (ws.gui.email_field.value and ws.gui.token_field.value)\n",
        "\n",
        "\n",
        "def handler_start_launch(button):\n",
        "    if not (ws.gui.email_field.value and ws.gui.token_field.value):\n",
        "        return\n",
        "    ws.gui.state = State.launch\n",
        "    try:\n",
        "        ws.start()\n",
        "    except WebServerException as error:\n",
        "        show_exception(error)\n",
        "        ws.gui.state = State.idle\n",
        "        ws.gui.launch_button.disabled = not (ws.gui.email_field.value and ws.gui.token_field.value)\n",
        "\n",
        "\n",
        "def handler_stop_launch(button):\n",
        "    ws.gui.state = State.idle\n",
        "    try:\n",
        "        ws.stop()\n",
        "    except WebServerException as error:\n",
        "        show_exception(error)\n",
        "        ws.gui.state = State.idle\n",
        "        ws.gui.launch_button.disabled = not (ws.gui.email_field.value and ws.gui.token_field.value)\n",
        "\n",
        "\n",
        "class WebServerGUI:\n",
        "    email_field: Text\n",
        "    token_field: Password\n",
        "    launch_button: Button\n",
        "    stop_button: Button\n",
        "\n",
        "    def __init__(self):\n",
        "        self.email_field = Text(placeholder=\"Введите E-mail\", disabled=True)\n",
        "        self.token_field = Password(placeholder=\"Введите Token\", disabled=True)\n",
        "        self.launch_button = Button(description=\"Запустить\", disabled=True)\n",
        "        self.stop_button = Button(description=\"Остановить\", disabled=True)\n",
        "\n",
        "        self.email_field.on_submit(handler_start_launch)\n",
        "        self.email_field.observe(handler_change_field, names=[\"value\"])\n",
        "\n",
        "        self.token_field.on_submit(handler_start_launch)\n",
        "        self.token_field.observe(handler_change_field, names=[\"value\"])\n",
        "\n",
        "        self.launch_button.on_click(handler_start_launch)\n",
        "        self.stop_button.on_click(handler_stop_launch)\n",
        "\n",
        "    @property\n",
        "    def state(self) -> Tuple[bool, bool, bool, bool]:\n",
        "        return (\n",
        "            not self.email_field.disabled,\n",
        "            not self.token_field.disabled,\n",
        "            not self.launch_button.disabled,\n",
        "            not self.stop_button.disabled\n",
        "        )\n",
        "\n",
        "    @state.setter\n",
        "    def state(self, value:State):\n",
        "        if value == State.idle:\n",
        "            self.email_field.disabled = False\n",
        "            self.token_field.disabled = False\n",
        "            self.launch_button.disabled = not (self.email_field.value and self.token_field)\n",
        "            self.stop_button.disabled = True\n",
        "        elif value == State.launch:\n",
        "            self.email_field.disabled = True\n",
        "            self.token_field.disabled = True\n",
        "            self.launch_button.disabled = True\n",
        "            self.stop_button.disabled = False\n",
        "\n",
        "    def display(self):\n",
        "        self.state = State.idle\n",
        "        widget.display(\n",
        "            VBox(\n",
        "                [\n",
        "                    self.email_field,\n",
        "                    self.token_field,\n",
        "                    HBox([self.launch_button, self.stop_button]),\n",
        "                ]\n",
        "            )\n",
        "        )\n",
        "\n",
        "\n",
        "class WebServer:\n",
        "    _branch:str = \"dev\"\n",
        "    _domain_prefix:str = \"dev.\"\n",
        "    _path:Path = Path(os.path.abspath(os.getcwd()))\n",
        "    _url:str = \"\"\n",
        "    _data:dict = {}\n",
        "    _proc:str\n",
        "\n",
        "    gui:WebServerGUI = WebServerGUI()\n",
        "\n",
        "    def _auth(self):\n",
        "        try:\n",
        "            response = requests.post(\n",
        "                f\"{TERRA_SERVER_API % self._domain_prefix}/login/\",\n",
        "                json={\"email\": self.gui.email_field.value, \"user_token\": self.gui.token_field.value},\n",
        "            )\n",
        "        except Exception as error:\n",
        "            raise WebServerException(error)\n",
        "\n",
        "        if not response.ok:\n",
        "            try:\n",
        "                response.raise_for_status()\n",
        "            except requests.exceptions.HTTPError as error:\n",
        "                raise WebServerException(error)\n",
        "\n",
        "        data = response.json()\n",
        "        if not data.get(\"success\"):\n",
        "            raise WebServerException(data.get(\"error\"))\n",
        "\n",
        "        self._url = data.get(\"data\", {}).get(\"url\", \"\")\n",
        "        if not self._url:\n",
        "            raise WebServerException(\"Не определен URL для пользовательского интерфейса\")\n",
        "\n",
        "        self._data = data.get(\"data\", {})\n",
        "\n",
        "    def _download(self):\n",
        "        _terra_path = Path(self._path, TERRA_DIRECTORY)\n",
        "        _repo_kwargs = {}\n",
        "        if self._branch:\n",
        "            _repo_kwargs.update({\"branch\": self._branch})\n",
        "        shutil.rmtree(_terra_path, ignore_errors=True)\n",
        "        try:\n",
        "            Repo.clone_from(TERRA_REPOSITORY, _terra_path, **_repo_kwargs)\n",
        "        except Exception as error:\n",
        "            raise WebServerException(error)\n",
        "\n",
        "    def _prepare(self):\n",
        "        _terra_path = Path(self._path, TERRA_DIRECTORY)\n",
        "        for name, info in self._data.get(\"create\", {}).items():\n",
        "            with open(Path(_terra_path, info.get(\"name\")), \"w\") as _file_ref:\n",
        "                _file_ref.write(info.get(\"data\"))\n",
        "\n",
        "        with open(Path(self._path, TERRA_DIRECTORY, \".env\"), \"a\") as _env_file_ref:\n",
        "            _env_file_ref.write(f\"USER_URL={self._url}\\n\")\n",
        "            _env_file_ref.write(f\"USER_EMAIL={self.gui.email_field.value}\\n\")\n",
        "            _env_file_ref.write(f\"USER_TOKEN={self.gui.token_field.value}\\n\")\n",
        "\n",
        "    def init(self):\n",
        "        self.gui.display()\n",
        "\n",
        "    def start(self):\n",
        "        self._auth()\n",
        "        self._download()\n",
        "        self._prepare()\n",
        "        print(f\"\\033[1;32mДля начала работы перейдите по следующей ссылке {self._url}\\033[0m\")\n",
        "        self._proc = Popen([\"make\", \"-C\", Path(self._path, TERRA_DIRECTORY)], stdin=PIPE, stdout=PIPE, stderr=STDOUT)\n",
        "\n",
        "    def stop(self):\n",
        "        if not self._proc:\n",
        "            return\n",
        "        self._proc.terminate()\n",
        "\n",
        "\n",
        "%cd /content\n",
        "\n",
        "ws = WebServer()\n",
        "mounted = False\n",
        "\n",
        "try:\n",
        "    drive.mount(\"/content/drive\")\n",
        "    mounted = True\n",
        "except ValueError as error:\n",
        "    print(\"Не удалось подключиться к GoogleDrive\")\n",
        "except KeyboardInterrupt as error:\n",
        "    pass\n",
        "\n",
        "if mounted:\n",
        "    output.clear()\n",
        "    print(\"Начало установки. Процесс может занять несколько минут, ожидайте...\")\n",
        "    !pip install git+https://github.com/aiuniver/terra_colab.git@dev &> /dev/null\n",
        "    print(\"Установка завершена!\")\n",
        "    sleep(1)\n",
        "    output.clear()\n",
        "    try:\n",
        "        ws.init()\n",
        "    except WebServerException as error:\n",
        "        show_exception(error)\n",
        "        ws.gui.state = State.idle\n"
      ],
      "execution_count": null,
      "outputs": []
    }
  ]
}